In [1]:
print("OK")

OK


In [3]:
%pwd

'd:\\Chatbot-Suresh-Beekhani\\research'

In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'd:\\'

In [8]:
 # For splitting text into chunks
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # HuggingFace embedding model for converting text to vectors
from langchain_qdrant import QdrantVectorStore  # type: ignore # For storing vectors in Qdrant # To load documents from the web (correct import statement)
import os  # Standard library for interacting with the operating system
from dotenv import load_dotenv
load_dotenv()


False

In [9]:

qdrant_api_key=os.environ.get('qdrant_api_key') 
os.environ["qdrant_api_key"] = qdrant_api_key
groq_API_KEY=os.environ.get('groq_API_KEY')
qdrant_url = "https://e92b3638-9cd8-43e3-9c5d-a049f560fab2.us-east4-0.gcp.cloud.qdrant.io"
collection_name = "skchatbot"
os.environ["groq_API_KEY"] = groq_API_KEY

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdfs(file_path, chunk_size, chunk_overlap):
    """Load a single PDF and split into chunks with custom separators."""
    
    # Load the single PDF file
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    # Define custom separators for splitting text
    separators = [
        "\n\n",  # Paragraph break
        "\n",    # Line break
        " ",     # Space
        ".",     # Full stop
        ",",     # Comma
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        ""        # Empty separator
    ]

    # Create a text splitter with the given chunk size, overlap, and separators
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
        separators=separators
    )
    
    # Split the documents into smaller chunks
    return text_splitter.split_documents(documents)

# Example of calling the function with the path to a single PDF file
file_path = r"D:\Chatbot-Suresh-Beekhani\data\surseh.pdf"  # Path to your single PDF file
doc_chunks = load_and_split_pdfs(file_path, chunk_size=500, chunk_overlap=10)


In [11]:
doc_chunks

[Document(metadata={'source': 'D:\\Chatbot-Suresh-Beekhani\\data\\surseh.pdf', 'page': 0}, page_content="Suresh Beekhani - Data Scientist and AI/ML Engineer\nOverview:\nSuresh Beekhani is a Data Scientist and AI/ML Engineer with over 3.5 years of industry \nexperience, specializing in Artiﬁcial Intelligence (AI), Generative AI, Natural Language \nProcessing (NLP), and Machine Learning (ML). He holds a Bachelor's degree in Computer \nScience and has extensive experience in developing and deploying machine learning models \nacross various industries.\nProfessional Experience:\nSkills and Expertise:"),
 Document(metadata={'source': 'D:\\Chatbot-Suresh-Beekhani\\data\\surseh.pdf', 'page': 0}, page_content='Generative AI Engineer at Fiverr (Dec 2023 – Present):\nSuresh collaborates with global clients, providing expertise in predictive modeling, NLP, \ncomputer vision, and deep learning. He is proﬁcient in Python, working with Large \nLanguage Models (LLMs), LLMOps, vector databases, and fr

In [12]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Get the API key from environment variables
api_key = os.getenv('google_api_key')
os.environ["GOOGLE_API_KEY"] = api_key

def get_google_embeddings():
    """Download and return the Google Generative AI embeddings."""
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    return embeddings

d:\Chatbot-Suresh-Beekhani\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
embeddings = get_google_embeddings()

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 768


In [17]:
# Initialize the QdrantVectorStore
qdrant = QdrantVectorStore.from_documents(
    documents=doc_chunks,              # List of Document objects to be stored in the vector store
    embedding=embeddings,              # Embedding model used to convert documents into vectors
    url=qdrant_url,                    # URL for the Qdrant service
    api_key=qdrant_api_key,            # API key for accessing the Qdrant service
    collection_name=collection_name    # Name of the collection to store the vectors in
)

In [18]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name= collection_name,
    url=qdrant_url,
    api_key=qdrant_api_key
)

In [19]:
retriever = qdrant.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [20]:
retrieved_docs = retriever.invoke("Suresh beekhani ?")

In [21]:
retrieved_docs

[Document(metadata={'source': 'D:\\Chatbot-Suresh-Beekhani\\data\\surseh.pdf', 'page': 0, '_id': '38ccfc87-70eb-4f5b-bb36-595131beb73c', '_collection_name': 'skchatbot'}, page_content="Suresh Beekhani - Data Scientist and AI/ML Engineer\nOverview:\nSuresh Beekhani is a Data Scientist and AI/ML Engineer with over 3.5 years of industry \nexperience, specializing in Artiﬁcial Intelligence (AI), Generative AI, Natural Language \nProcessing (NLP), and Machine Learning (ML). He holds a Bachelor's degree in Computer \nScience and has extensive experience in developing and deploying machine learning models \nacross various industries.\nProfessional Experience:\nSkills and Expertise:"),
 Document(metadata={'source': 'D:\\Chatbot-Suresh-Beekhani\\data\\surseh.pdf', 'page': 3, '_id': '73d07453-2d48-4916-a352-66426dd2622b', '_collection_name': 'skchatbot'}, page_content='Certiﬁcations:\nFor more information or to discuss potential collaborations, please contact Suresh Beekhani \nat sureshbeekhani2

In [22]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    api_key=groq_API_KEY,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [23]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define the system instruction
system_prompt = (
    "You are a concise and informative question-answering assistant. "
    "Provide accurate answers using the given context. "
    "If the answer is unknown, acknowledge it honestly. "
    "Keep responses brief, ideally within three sentences.\n\n"
    "{context}"
)

# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


In [24]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [25]:
response = rag_chain.invoke({"input": "Who is Suresh beekhan"})
print(response["answer"])

Suresh Beekhani is a Data Scientist and AI/ML Engineer with over 3.5 years of industry experience, specializing in Artificial Intelligence, Generative AI, Natural Language Processing, and Machine Learning. He has a Bachelor's degree in Computer Science and has worked on various projects across different industries. He is currently working as a Generative AI Engineer at Fiverr and a Machine Learning Engineer at Upwork.
